In [1]:
using CSV
using DataFrames
using Dates
using Gadfly

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20201221_0.csv"));
data = select(rawdata,
    :hospital_name => ByRow(titlecase) => :hospital,
    :collection_week => :date,
    :all_adult_hospital_inpatient_beds_7_day_avg => ByRow(x -> (ismissing(x) || x == -999999) ? missing : x) => :beds_allbeds,
    :total_icu_beds_7_day_avg => ByRow(x -> (ismissing(x) || x == -999999) ? missing : x) => :beds_icu,
);
data.beds_acute = data.beds_allbeds - data.beds_icu
sort!(data, [:hospital, :date]);

In [4]:
start_date = Date(2020,  7, 1)
end_date   = Date(2020, 11, 1)
date_range = collect(start_date : Day(1) : end_date);

In [5]:
function mean_capacity(xs, dates)
    xs = [x for (x,d) in zip(xs,dates) if d in date_range]
    if all(ismissing.(xs))
        return 0
    else
        xs = filter(x -> !ismissing(x), xs)
        return round(Int, sum(xs) / length(xs))
    end
end;

function max_capacity(xs)
    if all(ismissing.(xs))
        return 0
    else
        return maximum(skipmissing(xs))
    end
end;

function min_capacity(xs)
    if all(ismissing.(xs))
        return 0
    else
        return minimum(skipmissing(xs))
    end
end;

In [6]:
capacity_data = combine(groupby(data, :hospital), [
    :beds_icu => min_capacity => :beds_icu_min,
    :beds_acute => min_capacity => :beds_acute_min,
    :beds_allbeds => min_capacity => :beds_allbeds_min,

    (:beds_icu, :date) => mean_capacity => :beds_icu_mean,
    (:beds_acute, :date) => mean_capacity => :beds_acute_mean,
    (:beds_allbeds, :date) => mean_capacity => :beds_allbeds_mean,
        
    :beds_icu => max_capacity => :beds_icu_max,
    :beds_acute => max_capacity => :beds_acute_max,
    :beds_allbeds => max_capacity => :beds_allbeds_max,
]);

In [7]:
capacity_names_icu = [:beds_icu_min, :beds_icu_mean, :beds_icu_max];
capacity_names_acute = [:beds_acute_min, :beds_acute_mean, :beds_acute_max];
capacity_names_allbeds = [:beds_allbeds_min, :beds_allbeds_mean, :beds_allbeds_max];

In [8]:
filter!(row -> row.beds_icu_max + row.beds_acute_max + row.beds_allbeds_max > 0, capacity_data);

In [9]:
sort!(capacity_data, :hospital);

In [10]:
hospitals = Vector(capacity_data.hospital)
N = length(hospitals)
@show N;
@show hospitals;

N = 4693


hospitals = ["Abbeville Area Medical Center", "Abbeville General Hospital", "Abbott Northwestern Hospital", "Abilene Regional Medical Center", "Abington Memorial Hospital", "Abraham Lincoln Memorial Hospital", "Abrazo Arrowhead Campus", "Abrazo Central Campus", "Abrazo Scottsdale Campus", "Abrazo West Campus", "Abrom Kaplan Memorial Hospital", "Acadia General Hospital", "Acadia St Landry", "Acadian Medical Center", "Acmh Hospital", "Acuity Specialty Hospital Of Morgantown", "Acuity Specialty Hospital Of New Jersey", "Acuity Specialty Hospital Of Southern New Jersey", "Acuity Specialty Hospital Ohio Valley", "Ad Hospital East, Llc", "Adair County Memorial Hospital", "Adams County Regional Medical Center", "Adams Memorial Hospital", "Adena Pike Medical Center", "Adena Regional Medical Center", "Adirondack Medical Center - Saranac Lake", "Admin De Servicios Medicos  Puerto Ric", "Advanced Care Hospital Of Montana", "Advanced Care Hospital Of Southern New Mexico Llc", "Advanced Care Hospit

aston Campus", "St Luke'S Hospital - Lehighton", "St Luke'S Hospital - Monroe Campus", "St Luke'S Hospital - Sacred Heart Campus", "St Luke'S Hospital - Upper Bucks Campus", "St Luke'S Hospital At The Vintage", "St Luke'S Hospital Bethlehem", "St Luke'S Jerome", "St Luke'S Magic Valley Rmc", "St Luke'S Mccall", "St Luke'S Medical Center", "St Luke'S Miners Memorial Hospital", "St Luke'S Nampa Medical Center", "St Luke'S Patients Medical Center", "St Luke'S Regional Medical Center", "St Luke'S Sugar Land Hospital", "St Luke'S The Woodlands Hospital", "St Luke'S Warren Hospital", "St Luke'S Wood River Medical Center", "St Lukes Hospital", "St Lukes Hospital Of Kansas City", "St Lukes Lakeside Hospital", "St Lukes Regional Medical Center", "St Margarets Hospital", "St Mark'S Hospital", "St Marks Medical Center", "St Martin Hospital", "St Mary Medical Center", "St Mary Medical Center Inc", "St Mary Mercy Hospital", "St Mary'S General Hospital", "St Mary'S Good Samaritan Hospital", "St Mary

In [11]:
capacity_data

,hospital,beds_icu_min,beds_acute_min,beds_allbeds_min,beds_icu_mean,beds_acute_mean,beds_allbeds_mean,beds_icu_max,beds_acute_max,beds_allbeds_max
,String,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64
1,Abbeville Area Medical Center,0.0,10.6,10.6,0,16,16,0.0,18.0,18.0
2,Abbeville General Hospital,6.4,27.0,34.0,7,31,38,7.0,31.0,38.0
3,Abbott Northwestern Hospital,42.4,376.2,429.3,51,614,665,63.3,642.4,693.1
4,Abilene Regional Medical Center,9.3,51.4,66.0,14,68,82,21.3,83.1,104.4
5,Abington Memorial Hospital,89.0,399.0,488.0,89,432,521,89.0,436.0,525.0
6,Abraham Lincoln Memorial Hospital,0.0,25.0,25.0,0,25,25,0.0,25.0,25.0
7,Abrazo Arrowhead Campus,38.0,125.0,164.8,39,145,187,59.3,192.3,251.6
8,Abrazo Central Campus,23.6,76.3,104.3,29,81,109,34.0,97.2,125.3
9,Abrazo Scottsdale Campus,12.3,37.7,51.7,14,46,60,14.0,55.1,69.1


In [12]:
capacity_data_long = stack(capacity_data, Not([:hospital]), variable_name=:capacity_name, value_name=:capacity_value);

In [13]:
capacity_data_long_icu = filter(row -> Symbol(row.capacity_name) in capacity_names_icu, capacity_data_long);
capacity_data_long_acute = filter(row -> Symbol(row.capacity_name) in capacity_names_acute, capacity_data_long);
capacity_data_long_allbeds = filter(row -> Symbol(row.capacity_name) in capacity_names_allbeds, capacity_data_long);

In [14]:
capacity_data_output = select(capacity_data, :hospital, :beds_icu_mean => :capacity_icu, :beds_acute_mean => :capacity_acute, :beds_allbeds_mean => :capacity_allbeds);

In [15]:
capacity_data_output |> CSV.write("../data/capacity_hhs.csv");